In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X = dataset.iloc[:,3:13]
y= dataset.iloc[:,13]

In [6]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [7]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [13]:
## Feature Engineering 
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

In [15]:

X=X.drop(['Geography','Gender'],axis=1)

In [17]:
## Concatenate 
X=pd.concat([X,geography,gender],axis=1)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train  , y_test = train_test_split(X,y,test_size=0.20,random_state=0)

In [20]:
#Feeature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [21]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [22]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ...,  1.75486502,
        -0.57369368,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.5698444 ,
        -0.57369368, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.5698444 ,
        -0.57369368,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ..., -0.5698444 ,
         1.74309049, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ...,  1.75486502,
        -0.57369368, -1.09168714]])

In [24]:
X_train.shape

(8000, 11)

In [27]:
## Create The ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,ReLU,PReLU,ELU
from tensorflow.keras.layers import Dropout


In [28]:
Classifier = Sequential()

In [31]:
## Adding Input layer
Classifier.add(Dense(units=11,activation='relu'))

In [32]:
Classifier.add(Dense(units=7,activation=
                    'relu'))

In [35]:
Classifier.add(Dense(units=6,activation='relu'))

In [36]:
Classifier.add(Dense(1,activation='sigmoid'))

In [37]:
Classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [42]:
## Early Stopping
import tensorflow as tf
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=20,
    verbose=1
    ,mode='auto',
    baseline=None,
    restore_best_weights=False

)



In [43]:
model_history = Classifier.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=1000,callbacks=early_stopping)

Epoch 1/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8810 - loss: 0.2945 - val_accuracy: 0.8455 - val_loss: 0.3704
Epoch 2/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8748 - loss: 0.2982 - val_accuracy: 0.8504 - val_loss: 0.3663
Epoch 3/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8719 - loss: 0.3022 - val_accuracy: 0.8512 - val_loss: 0.3667


In [50]:
dat=pd.DataFrame(model_history.history)

In [51]:
dat

,accuracy,loss,val_accuracy,val_loss
0,0.869565,0.311814,0.845513,0.370425
1,0.868632,0.312061,0.850435,0.366280
2,0.871431,0.310993,0.851193,0.366704


In [52]:
y_pred = Classifier.predict(X_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [53]:
y_pred = (y_pred >= 0.5)

In [55]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
cm

array([[1528,   67],
       [ 221,  184]], dtype=int64)

In [56]:
score = accuracy_score(y_test,y_pred)

In [57]:
score

0.856

In [59]:
## Get The Weight
Classifier.get_weights()

[array([[ 1.7137246e-02, -5.3616717e-02, -4.6992935e-02, -2.0323072e-01,
          1.3016137e-01,  1.6296681e-02,  3.6163187e-01,  1.8753338e-01,
         -4.0265191e-02, -1.7978859e-01,  6.7448273e-02],
        [-5.6661624e-01, -2.1735935e-01,  4.7316957e-01,  1.5804239e-01,
         -2.4770798e-01, -2.2455126e-01, -8.7718904e-01, -5.6086022e-01,
          4.6028253e-02, -2.7181131e-01,  8.5259712e-01],
        [-1.5315743e-01,  9.7331822e-02, -4.9074050e-02,  9.7579263e-02,
          8.1325865e-01,  1.9380720e-01,  1.2799349e-01,  5.8305226e-02,
         -2.1154241e-01,  8.1046081e-01,  1.6786829e-01],
        [-3.4181741e-01, -8.9534454e-02, -4.3220493e-01,  1.4837167e-01,
          3.7020388e-01,  6.6518396e-01, -9.1539323e-02,  2.2302078e-01,
          5.8555681e-01, -3.8366657e-01, -4.2227954e-01],
        [-2.0754378e-01, -1.0233090e+00, -1.1961619e+00,  4.4225402e-02,
          2.5493768e-01,  6.8715692e-01,  2.0717460e-01,  5.2360111e-01,
          1.0559356e+00,  9.1998965e-0